In [1]:
import ipyrad.analysis as ipa
import toytree
import toyplot

In [2]:
print('ipyrad', ipa.__version__)
print('toytree', toytree.__version__)
! treemix --version | grep 'TreeMix v. '

ipyrad 0.9.43
toytree 1.1.2
TreeMix v. 1.13


In [3]:
# the path to your HDF5 formatted snps file
data = "T_ae_8rm_v9.snps.hdf5"

In [48]:
imap = {
#"ref": ["reference"],
"Inam": ["T_ae_T14205_In", "T_ae_A7968_In", "T_ae_A8087_In", "T_ae_28095_In","T_ae_75520_In","T_ae_T12313_pu", "T_ae_T13219_pu", "T_ae_A3559_pu", "T_ae_A439_pu", "T_ae_82491_pu", "T_ae_82606_pu", "T_ae_82608_pu", "T_ae_A21562_pu", "T_ae_A2720_pu", "T_ae_A2833_pu"], 
"Rondonia": ["T_ae_T3376_jigu","T_ae_T3237_jigu", "T_ae_T3260_jigu", "T_ae_A317_jigu", "T_ae_A324_jigu","T_ae_T13278_ma", "T_ae_A458_ma", "T_ae_T2166_ma", "T_ae_T2207_ma", "T_ae_T4355_ma", "T_ae_A509_ma", "T_ae_J249_ma", "T_ae_J252_ma", "T_ae_J261_ma", "T_ae_J298_ma","T_ae_J319_roar", "T_ae_J419_roar", "T_ae_J678_roar", "T_ae_J683_roar"],
"ArSuTa": ["T_ae_80508_arsu", "T_ae_80716_arsu", "T_ae_81278_arsu",   "T_ae_A522_ma",  "T_ae_86147_arsu", "T_ae_86229_arsu",  "T_ae_J598_arsu", "T_ae_J616_arsu","T_ae_86569_arsu","T_ae_81338_arsu","T_ae_T14601_suta", "T_ae_T16625_suta", "T_ae_T16704_suta", "T_ae_85274_suta", "T_ae_A15069_suta", "T_ae_T742_suta", "T_ae_T10229_suta", "T_ae_T19436_suta", "T_ae_A16041_suta","T_ae_T7941_suta"],
"Para": ["T_ae_A15279_pa", "T_ae_T10679_pa", "T_ae_T13575_pa", "T_ae_T17307_pa", "T_ae_T19420_pa", "T_ae_T8268_pa","T_ae_T24591_suta"]
}

# minimum % of samples that must be present in each SNP from each group
minmap = {i: 0.5 for i in imap}

In [53]:
tmx1 = ipa.treemix(
    data=data, 
    imap=imap,
    minmap=minmap,
    root="Para",
    global_=True,
    m=2
)

Samples: 61
Sites before filtering: 1641207
Filtered (indels): 0
Filtered (bi-allel): 45665
Filtered (mincov): 0
Filtered (minmap): 1211661
Filtered (combined): 1228773
Sites after filtering: 412434
Sites containing missing values: 411661 (99.81%)
Missing values in SNP matrix: 4617921 (18.36%)
subsampled 33676 unlinked SNPs


In [54]:
# print the command string that will be called and run it
print(tmx1.command)
tmx1.run()

/home/lmusher/array1/miniconda3/envs/treemix/bin/treemix -i /array1/lmusher/rio_roosevelt_outfiles/T_ae_8rm_v9_outfiles/analysis-treemix/test.treemix.in.gz -o /array1/lmusher/rio_roosevelt_outfiles/T_ae_8rm_v9_outfiles/analysis-treemix/test -m 2 -seed 192292415 -root Para -global


In [55]:
# draw the resulting tree
tmx1.draw_tree();
tmx1.results.llik

84.9655

<svg class="toyplot-canvas-Canvas" height="275.0px" id="t8d4e7167d74348c8b8a65c24532ace3b" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 290.0 275.0" width="290.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Rondonia ArSuTa Inam Para 0.000 0.008 0.017

In [56]:
# draw the covariance matrix
tmx1.draw_cov();

<svg class="toyplot-canvas-Canvas" height="400.0px" id="te441072f55c74714b241f5f884578d48" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 400.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0.006836 -0.002588 -0.001511 -0.002737 Para -0.002588 0.003060 -0.000563 0.000091 Inam -0.001511 -0.000563 0.001969 0.000104 ArSuTa -0.002737 0.000091 0.000104 0.002542 Rondonia Para Inam ArSuTa Rondonia

#We are now randomly sampling 1snp per locus, over 100 iterations to see how likelihood changes
#We are also sampling datasets that vary randomly in the percent of missing data ranging from 50% complete to 95% complete
#We do this for m = 1:5

In [9]:
import random
boots = range(250)

In [10]:
tests0 = {}
samp0 = {}
for i in boots:
    minSamp = random.randrange(50,95)/100
    # init a treemix analysis object with a random (no) seed
    tmx = ipa.treemix(
        data=data, 
        imap=imap,
        minmap={i: minSamp for i in imap},
        root="Para",
        global_=True,
        m=0,
        quiet=True
    )
    #print(minSamp),
    # run model fit
    tmx.run()
    tests0[i] = tmx.results.llik
    samp0[i] = minSamp

In [11]:
tests1 = {}
samp1 = {}
for i in boots:
    minSamp = random.randrange(50,95)/100
    # init a treemix analysis object with a random (no) seed
    tmx = ipa.treemix(
        data=data, 
        imap=imap,
        minmap={i: minSamp for i in imap},
        root="Para",
        global_=True,
        m=1,
        quiet=True
    )
    #print(minSamp),
    # run model fit
    tmx.run()
    tests1[i] = tmx.results.llik
    samp1[i] = minSamp

In [12]:
tests2 = {}
samp2 = {}
for i in boots:
    minSamp = random.randrange(50,95)/100
    # init a treemix analysis object with a random (no) seed
    tmx = ipa.treemix(
        data=data, 
        imap=imap,
        minmap={i: minSamp for i in imap},
        root="Para",
        global_=True,
        m=2,
        quiet=True
    )
    #print(minSamp),
    # run model fit
    tmx.run()
    tests2[i] = tmx.results.llik
    samp2[i] = minSamp

In [13]:
tests3 = {}
samp3 = {}
for i in boots:
    minSamp = random.randrange(50,95)/100
    # init a treemix analysis object with a random (no) seed
    tmx = ipa.treemix(
        data=data, 
        imap=imap,
        minmap={i: minSamp for i in imap},
        root="Para",
        global_=True,
        m=3,
        quiet=True
    )
    #print(minSamp),
    # run model fit
    tmx.run()
    tests3[i] = tmx.results.llik
    samp3[i] = minSamp

In [14]:
import numpy

test0 = numpy.mean([tests0[i] for i in boots])
test1 = numpy.mean([tests1[i] for i in boots])
test2 = numpy.mean([tests2[i] for i in boots])
test3 = numpy.mean([tests3[i] for i in boots])

tests = [test0, test1, test2, test3]

In [15]:
# plot the likelihood for different values of m
toyplot.plot(
    range(len(tests)),
    [tests[i] for i in range(len(tests))],
    width=350, 
    height=275,
    stroke_width=4,
    xlabel="# admixture edges",
    ylabel="ln(likelihood)",
    color="steelblue"
);

<svg class="toyplot-canvas-Canvas" height="275.0px" id="t08bb8ae8f8654e3bb2ed7d4ed1842ac8" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 275.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 # admixture edges 65 70 75 80 ln(likelihood)

In [16]:
# import pandas as pd
import pandas as pd 

# list of strings 
test0 = [tests0[i] for i in boots]
test1 = [tests1[i] for i in boots]
test2 = [tests2[i] for i in boots]
test3 = [tests3[i] for i in boots]
samps0 = [samp0[i] for i in boots]
samps1 = [samp1[i] for i in boots]
samps2 = [samp2[i] for i in boots]
samps3 = [samp3[i] for i in boots]

# Calling DataFrame constructor on list 
df = pd.DataFrame(list(zip(test0, samps0, test1, samps1, test2, samps2, test3, samps3)), 
               columns =['m0', 's0', 'm1', 's1', 'm2', 's2', 'm3', 's3']) 

df.to_csv(r'./analysis-treemix/G_cy_nadmix_tests.csv')

In [17]:
df.head()

,m0,s0,m1,s1,m2,s2,m3,s3
0,69.989,0.92,84.317,0.58,85.079,0.51,84.181,0.60
1,55.836,0.60,84.142,0.61,82.477,0.71,84.206,0.65
2,67.727,0.77,82.238,0.72,72.214,0.91,75.087,0.88
3,68.185,0.75,84.674,0.52,85.032,0.50,72.592,0.92
4,44.125,0.55,76.615,0.88,82.244,0.74,81.764,0.80


In [27]:
# a gridded canvas to plot trees on 
canvas = toyplot.Canvas(width=1200, height=200)
counts = 0
# iterate over multiple set of SNPs
for i in [0.5, 0.6, 0.7, 0.8, 0.9]:
    
    # init a treemix analysis object with a random (no) seed
    tmx = ipa.treemix(
        data=data, 
        imap=imap,
        minmap={j: i for j in imap},
        root="Para,ArSuTa",
        global_=True,
        m=1,
        quiet=True
    )
    
    # run model fit
    tmx.run()

    # select a plot grid axis and add tree to axes
    axes = canvas.cartesian(grid=(1, 6, counts))
    tmx.draw_tree(axes)
    counts = counts+1

<svg class="toyplot-canvas-Canvas" height="200.0px" id="tb6a4a917660d4c048b9a7c0a388084cc" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1200.0 200.0" width="1200.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Rondonia Inam ArSuTa Para 0.000 0.006 0.011 Rondonia Inam ArSuTa Para 0.000 0.005 0.010 Inam Rondonia Para ArSuTa 0.000 0.006 0.011 Inam Rondonia Para ArSuTa 0.000 0.006 0.011 Rondonia ArSuTa Para Inam 0.000 0.005 0.009